In [4]:
import sys
sys.path.append('../_amigocloud')

In [5]:
from amigocloud import AmigoCloud
import pandas as pd
from shapely import wkb

import geopandas as gpd

from datetime import datetime, timedelta
from shapely import wkb
from shapely.geometry import Polygon, MultiPolygon

In [6]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [7]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [81]:
path_xlsx_cels = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2024\TELEFONOS.xlsx'

In [82]:
df_cels = pd.read_excel(path_xlsx_cels)

In [83]:
df_cels

,institucion,cod_ca,ca,telefono_1,telefono_2,telefono_3
0,32,8523,MIRANDA PEREZ NICASIO,NaN,NaN,NaN
1,50,4844,TARRADELLES MARTINEZ DIETER JORGE,78188288.0,NaN,NaN
2,14,41532,OVANDO JIMENEZ EDUARDO,77092101.0,NaN,NaN
3,152,2100,CONDORI MAMANI JUAN,71670475.0,NaN,NaN
4,66,7821,LEON MONDAQUE ELISA,67813629.0,NaN,NaN
...,...,...,...,...,...,...
1559,14,30932,GUZMAN ORELLANA JHOANA GRISELDA,70955542.0,NaN,NaN
1560,140,42073,TAPIA SEGOVIA ENRIQUE,NaN,NaN,NaN
1561,68,2042,CKOLO ESCALERA ABIGAIL,63574866.0,NaN,NaN
1562,43,3460,ESPINOZA MENDOZA SANTIAGO,76076797.0,NaN,NaN


In [84]:
def convertir_wkb(wkb_data):
    geom = wkb.loads(wkb_data, hex=True)
    # Si la geometría es un MultiPolygon con un solo polígono, conviértelo a Polígono
    if isinstance(geom, MultiPolygon) and len(geom.geoms) == 1:
        geom = geom.geoms[0]
    return geom

In [85]:
semana = 27

In [86]:
query = f"select cast(NULLIF(split_part(insp.semana_3ra, 'S', 2), '') as integer) as sem_reg, institucion, cast(split_part(prop.propiedad, ' / ', 1) as integer) unidad_01, split_part(prop.propiedad, ' / ', 2) unidad_02, cast(split_part(insp.canhero, ' / ', 1) as integer) unidad_03, split_part(insp.canhero, ' / ',2) unidad_04, lote.lote unidad_05, lote.superficie area, lote.fecha_siembra::date fecha_ini, 'SIEMBRA 2024' origen, lote.geometria geometry from dataset_307560 insp inner join dataset_307562 prop on insp.amigo_id = prop.insp_ref_prop inner join dataset_307563 lote on prop.amigo_id = lote.prop_ref_lote where insp.estado_3ra_insp = 'APROBADO' and (lote.estado_de_siembra = 'CAÑA NACIDA' or lote.estado_de_siembra = 'CAÑA NO NACIDA') and cast(NULLIF(split_part(insp.semana_3ra, 'S', 2), '') as integer) = {semana}"
query_consulta = {'query': query}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_consulta)
data = select['data']

In [87]:
data

[{'sem_reg': 27,
  'institucion': 3,
  'unidad_01': 89,
  'unidad_02': 'EL TOTAI--MOLINA',
  'unidad_03': 8684,
  'unidad_04': 'MOLINA PORCEL JUAN CARLOS',
  'unidad_05': 'L6',
  'area': 0.43,
  'fecha_ini': '2024-05-20',
  'origen': 'SIEMBRA 2024',
  'geometry': '0106000020E61000000100000001030000000100000005000000E2A2E6CD018F4FC07CCE7BE4722231C0642285E4E88E4FC0E8E76466722231C012419C08E88E4FC028F61BEC912231C05AE2EF75018F4FC07C902316922231C0E2A2E6CD018F4FC07CCE7BE4722231C0'},
 {'sem_reg': 27,
  'institucion': 138,
  'unidad_01': 1650,
  'unidad_02': 'ASUBISITO--BERRIOS HUMBERTO',
  'unidad_03': 41599,
  'unidad_04': 'LOPEZ TURIHUANO JUAN CARLOS',
  'unidad_05': 'L2',
  'area': 5.92,
  'fecha_ini': '2024-05-23',
  'origen': 'SIEMBRA 2024',
  'geometry': '0106000020E610000001000000010300000001000000090000003DF27D5F5CB84FC0856123C57D1831C0F5D6A5F65AB84FC02A223B7B7D1831C03E74637B53B84FC03086A091891731C09AB8C1EA7DB84FC045A453DB5C1731C0924E15FC81B84FC04B347D27691731C057CEF1FB85B84FC06CEF84D0

In [88]:
df = pd.DataFrame(data)
df['geometry'] = df['geometry'].apply(convertir_wkb)
data = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')
gdf_utm = data.to_crs('EPSG:32720')
gdf_utm['fecha_ini'] = pd.to_datetime(gdf_utm['fecha_ini'])
now = datetime.now()
gdf_utm['dias'] = (now - gdf_utm['fecha_ini']).dt.days
gdf_utm['fecha_ini'] = gdf_utm['fecha_ini'].dt.strftime('%Y-%m-%d')

In [89]:
gdf_utm.rename(columns={'institucion': 'inst'}, inplace=True)

In [90]:
len(gdf_utm)

41

In [91]:
gdf_utm

,sem_reg,inst,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,area,fecha_ini,origen,geometry,dias
0,27,3,89,EL TOTAI--MOLINA,8684,MOLINA PORCEL JUAN CARLOS,L6,0.43,2024-05-20,SIEMBRA 2024,"POLYGON ((487529.255 8105554.786, 487610.120 8...",58
1,27,138,1650,ASUBISITO--BERRIOS HUMBERTO,41599,LOPEZ TURIHUANO JUAN CARLOS,L2,5.92,2024-05-23,SIEMBRA 2024,"POLYGON ((453154.481 8109808.605, 453159.057 8...",55
2,27,138,823,EL COLORADO II--JUAN CONDORI,41599,LOPEZ TURIHUANO JUAN CARLOS,L3.1,4.41,2024-05-29,SIEMBRA 2024,"POLYGON ((453706.829 8100003.702, 453772.778 8...",49
3,27,5,2141,LA CORRIENTE--SOTO BASILIO,1237,BALCAZAR GUTIERREZ JOSE IVER,L1,0.79,2024-04-22,SIEMBRA 2024,"POLYGON ((465456.027 8097189.931, 465397.503 8...",86
4,27,5,1092,BENJAMIN SOTO I--SOTO,13062,SOTO ARCE BACILIO,L4.1,2.82,2024-04-24,SIEMBRA 2024,"POLYGON ((460066.557 8114531.576, 459839.424 8...",84
5,27,138,2156,ASUBI--LOPEZ JUAN CARLOS,41599,LOPEZ TURIHUANO JUAN CARLOS,L1,11.01,2024-05-28,SIEMBRA 2024,"POLYGON ((453092.921 8110864.447, 453305.955 8...",50
6,27,81,637,SILES--CORDOVA,42239,CORDOVA OLGUIN LEONARDO,L1.1,13.29,2024-04-23,SIEMBRA 2024,"POLYGON ((487919.951 8098928.529, 487825.306 8...",85
7,27,138,1650,ASUBISITO--BERRIOS HUMBERTO,41599,LOPEZ TURIHUANO JUAN CARLOS,L3,1.11,2024-05-24,SIEMBRA 2024,"POLYGON ((453152.575 8109748.677, 453156.839 8...",54
8,27,5,2141,LA CORRIENTE--SOTO BASILIO,1237,BALCAZAR GUTIERREZ JOSE IVER,L7,1.19,2024-04-23,SIEMBRA 2024,"POLYGON ((465471.936 8097269.169, 465465.819 8...",85
9,27,5,83,LA ESQUINA--SOTO,13062,SOTO ARCE BACILIO,L7,1.50,2024-04-24,SIEMBRA 2024,"POLYGON ((487987.557 8101150.934, 487960.795 8...",84


In [92]:
# Realizar el merge de los dataframes
df_merged = gdf_utm.merge(df_cels[['cod_ca', 'telefono_1', 'telefono_2', 'telefono_3']], left_on='unidad_03', right_on='cod_ca', how='left')

# Eliminar la columna 'cod_ca' duplicada después del merge
df_merged.drop(columns=['cod_ca'], inplace=True)

In [93]:
df_merged['telefono_1'] = pd.to_numeric(df_merged['telefono_1'], errors='coerce').fillna(0).astype(int)
df_merged['telefono_2'] = pd.to_numeric(df_merged['telefono_2'], errors='coerce').fillna(0).astype(int)
df_merged['telefono_3'] = pd.to_numeric(df_merged['telefono_3'], errors='coerce').fillna(0).astype(int)

In [94]:
len(df_merged)

41

In [95]:
df_merged

,sem_reg,inst,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,area,fecha_ini,origen,geometry,dias,telefono_1,telefono_2,telefono_3
0,27,3,89,EL TOTAI--MOLINA,8684,MOLINA PORCEL JUAN CARLOS,L6,0.43,2024-05-20,SIEMBRA 2024,"POLYGON ((487529.255 8105554.786, 487610.120 8...",58,67808296,0,0
1,27,138,1650,ASUBISITO--BERRIOS HUMBERTO,41599,LOPEZ TURIHUANO JUAN CARLOS,L2,5.92,2024-05-23,SIEMBRA 2024,"POLYGON ((453154.481 8109808.605, 453159.057 8...",55,73393159,0,0
2,27,138,823,EL COLORADO II--JUAN CONDORI,41599,LOPEZ TURIHUANO JUAN CARLOS,L3.1,4.41,2024-05-29,SIEMBRA 2024,"POLYGON ((453706.829 8100003.702, 453772.778 8...",49,73393159,0,0
3,27,5,2141,LA CORRIENTE--SOTO BASILIO,1237,BALCAZAR GUTIERREZ JOSE IVER,L1,0.79,2024-04-22,SIEMBRA 2024,"POLYGON ((465456.027 8097189.931, 465397.503 8...",86,60963933,0,0
4,27,5,1092,BENJAMIN SOTO I--SOTO,13062,SOTO ARCE BACILIO,L4.1,2.82,2024-04-24,SIEMBRA 2024,"POLYGON ((460066.557 8114531.576, 459839.424 8...",84,68774263,75507168,74717794
5,27,138,2156,ASUBI--LOPEZ JUAN CARLOS,41599,LOPEZ TURIHUANO JUAN CARLOS,L1,11.01,2024-05-28,SIEMBRA 2024,"POLYGON ((453092.921 8110864.447, 453305.955 8...",50,73393159,0,0
6,27,81,637,SILES--CORDOVA,42239,CORDOVA OLGUIN LEONARDO,L1.1,13.29,2024-04-23,SIEMBRA 2024,"POLYGON ((487919.951 8098928.529, 487825.306 8...",85,0,0,0
7,27,138,1650,ASUBISITO--BERRIOS HUMBERTO,41599,LOPEZ TURIHUANO JUAN CARLOS,L3,1.11,2024-05-24,SIEMBRA 2024,"POLYGON ((453152.575 8109748.677, 453156.839 8...",54,73393159,0,0
8,27,5,2141,LA CORRIENTE--SOTO BASILIO,1237,BALCAZAR GUTIERREZ JOSE IVER,L7,1.19,2024-04-23,SIEMBRA 2024,"POLYGON ((465471.936 8097269.169, 465465.819 8...",85,60963933,0,0
9,27,5,83,LA ESQUINA--SOTO,13062,SOTO ARCE BACILIO,L7,1.50,2024-04-24,SIEMBRA 2024,"POLYGON ((487987.557 8101150.934, 487960.795 8...",84,68774263,75507168,74717794


In [96]:
df_merged['area'].sum()

180.9

In [216]:
#file_name = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2024\SHP\SIEMBRAS.shp'

# Exporta el GeoDataFrame como un shapefile
#gdf_utm.to_file(file_name)

In [97]:
file_name = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2024\SHP\SIEMBRAS_2024.shp'
gdf_siembras = gpd.read_file(file_name)
gdf_siembras = pd.concat([gdf_siembras, df_merged], ignore_index=True)
gdf_siembras.to_file(file_name)